## IMPORTING THE RULES SET

In [1]:
import pandas as pd

def replace_frozenset(x):
    modified_string = x.replace('frozenset','').replace('{','').replace('}', '').replace('(', '').replace(' ', '').replace(')', '').replace("'",'')
    return [item for item in modified_string.split(',')]
    
#version1: rules.csv
rulesDF = pd.read_csv("../csv_sources/parsed_rules.csv", converters={
    'antecedents': lambda x: replace_frozenset(x),
    'consequents': lambda y: replace_frozenset(y)
}).drop(['Unnamed: 0'], axis='columns')

rulesDF

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"[26000000, 28000012, 28000015]",[26000023],0.012571,0.079857,0.011857,0.943182,11.810864,0.010853,16.194514
1,"[26000000, 28000015, 26000023]",[28000012],0.012429,0.121500,0.011857,0.954023,7.852041,0.010347,19.107375
2,"[28000012, 26000000, 26000015]",[26000023],0.012643,0.079857,0.011071,0.875706,10.965910,0.010062,7.402968
3,"[26000015, 26000000, 26000023]",[28000012],0.012071,0.121500,0.011071,0.917160,7.548640,0.009605,10.604750
4,"[26000000, 28000015, 26000023]",[26000015],0.012429,0.255286,0.011000,0.885057,3.466929,0.007827,6.479014
...,...,...,...,...,...,...,...,...,...
7868,"[28000008, 26000029, 26000008]",[26000006],0.011714,0.154571,0.010857,0.926829,5.996123,0.009046,11.554190
7869,"[28000008, 26000006, 26000008]",[26000029],0.011714,0.037357,0.010857,0.926829,24.809961,0.010420,13.156119
7870,"[26000008, 26000029, 26000006]",[28000008],0.012929,0.244143,0.010857,0.839779,3.439703,0.007701,4.717591
7871,"[28000000, 26000029]",[26000006],0.012929,0.154571,0.010357,0.801105,5.182749,0.008359,4.250627


## IMPORTING THE DECKS

In [3]:
from pymongo import MongoClient
from bson import ObjectId
import json

client = MongoClient("mongodb://localhost:27017")
db = client.DataMining
collection = db.parsed_matches
matchesList = json.loads(json.dumps([card for card in collection.find({})], default=str))
client.close()

matchList = []

for match in matchesList:
    localList = [str(match['card1']['id']),str(match['card2']['id']),str(match['card3']['id']),str(match['card4']['id']),str(match['card5']['id']),str(match['card6']['id']),str(match['card7']['id']),str(match['card8']['id'])]
    doc = {'id': match['_id'], 'deck': localList}
    matchList.append(doc)
print(matchList)

[{'id': '6204ef4c0ed73f2ecf5cd3ca', 'deck': ['28000002', '26000023', '26000012', '26000036', '26000027', '28000011', '26000048', '26000015']}, {'id': '6204ef4c0ed73f2ecf5cd3cb', 'deck': ['26000016', '26000011', '26000015', '26000023', '26000012', '28000011', '28000001', '28000004']}, {'id': '6204ef4c0ed73f2ecf5cd3cc', 'deck': ['26000009', '26000015', '28000012', '28000008', '26000048', '26000035', '26000039', '27000007']}, {'id': '6204ef4c0ed73f2ecf5cd3cd', 'deck': ['26000017', '26000012', '26000024', '26000021', '26000022', '26000043', '26000020', '27000010']}, {'id': '6204ef4c0ed73f2ecf5cd3ce', 'deck': ['26000008', '28000001', '26000051', '26000064', '26000015', '26000014', '26000000', '26000040']}, {'id': '6204ef4c0ed73f2ecf5cd3cf', 'deck': ['28000007', '26000015', '28000012', '28000014', '26000009', '26000013', '26000048', '26000016']}, {'id': '6204ef4c0ed73f2ecf5cd3d0', 'deck': ['26000021', '28000000', '26000001', '28000008', '26000046', '26000005', '26000004', '27000006']}, {'id'

## CALCULATING THE SYNERGIES

In [4]:
#rule[0] is the antecedent
#rule[1] is the consequent
#rule[5] is the confidence
#rule[6] is the lift

from multiprocessing import Process
from itertools import combinations

print(f"{len(matchList)} decks to analyze")
analyzed_decks = 0

def analyze_deck(match):
    deck = match['deck']
    _id = match['id']
    n_rules = 0
    total_conf = 0
    total_lift = 0
    for i in range(1,8):
        combos = combinations(deck, i)
        for c in combos:
            conversion = sorted([x for x in c])
            found_match = [rule for rule in rulesDF.values if conversion == sorted(rule[0])]
            if len(found_match) > 0:
                total_conf+=found_match[5]
                total_lift+=found_match[6]
                n_rules+=1
    if n_rules > 0:
        avg_conf = total_conf / n_rules
        avg_lift = total_lift / n_rules
        collection.update({_id: ObjectId(_id)}, {"$set": {"conf_synergy": avg_conf, "lift_synergy": avg_lift}})
    return

active_processes = []

for match in matchList:
    analyzed_decks += 1
    # START OF THE DECK'S CALCULATION
    if len(active_processes) >= 4:
        to_join = active_processes.pop(0)
        to_join.join()
        print(f"concluded process, {analyzed_decks} analyzed")
        p = Process(target=analyze_deck, args=(match,))
        print(f"starting new out of bound process {match['id']}")
        p.start()
        active_processes.append(p)
        continue
    print(f"starting in bound process {match['id']}")
    p = Process(target=analyze_deck, args=(match,))
    p.start()
    active_processes.append(p)
    p.join() # RICORDA DI ELIMINARE QUESTA RIGA
    break

14000 decks to analyze
starting in bound process 6204ef4c0ed73f2ecf5cd3ca
